# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name:

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

1. Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import spacytextblob
import numpy as np
import html5lib
import pickle

# write a function to scrape the html of the url and pickle it
def scrape_and_pickle(url, filename):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html5lib')
    with open(filename, 'wb') as file:
        pickle.dump(soup, file)
scrape_and_pickle('https://www.theringer.com/2025/06/25/nba/cooper-flagg-maine-nba-draft', 'cooperflagg_theringer.pkl')

2. Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

In [21]:
import pickle
import spacy
from spacy.language import Language
from spacy.tokens import Doc
from textblob import TextBlob

# Manually define the extensions (what SpacyTextBlob usually does under the hood)
Doc.set_extension("polarity", getter=lambda doc: TextBlob(doc.text).sentiment.polarity, force=True)
Doc.set_extension("subjectivity", getter=lambda doc: TextBlob(doc.text).sentiment.subjectivity, force=True)

# Define the pipeline component manually
@Language.component("spacytextblob_manual")
def spacytextblob_manual(doc):
    return doc

# Load model and add our manual component
nlp = spacy.load("en_core_web_sm")
if "spacytextblob_manual" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob_manual", last=True)

# Read the pickled BeautifulSoup HTML object
def read_pickled_html(filename):
    with open(filename, 'rb') as file:
        soup = pickle.load(file)
    return soup

# Extract text from the BeautifulSoup object
def extract_text(soup):
    return soup.get_text()

# Analyze sentiment using TextBlob via spaCy extension
def analyze_sentiment(text):
    doc = nlp(text)
    return doc._.polarity, doc._.subjectivity

# Run it
soup = read_pickled_html('cooperflagg_theringer.pkl')
text = extract_text(soup)
sentiment = analyze_sentiment(text)
print(f"Polarity: {sentiment[0]}, Subjectivity: {sentiment[1]}")

Polarity: -0.25422942657328, Subjectivity: 0.6988888784344831


3. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?